### Conversation Summary Memory 
الكود بيبني نظام محادثة بين إنسان وذكاء اصطناعي، الذكاء الصناعي بيحتفظ `بملخص` المحادثة باستخدام ConversationSummaryMemory علشان يتذكر الحوار، بدل ما يخزن كل الكلام كامل. وكمان بيستخدم LLMChain عشان يربط الـ model بالـ prompt والذاكرة مع بعض

In [49]:
import openai
from openai import OpenAI
import langchain
import os
# Step 1: Load environment variables
%reload_ext dotenv
%dotenv
# Step 3: Access your OpenAI API key from environment variables
for key, value in os.environ.items():
    if key == "OPENAI_API_KEY":
        print(f"{key}: value")

OPENAI_API_KEY: value


In [50]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [51]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini', temperature=0,max_completion_tokens=50)
# response = chat.invoke("I have recently adopted a dog. Could you suggest some dog names?")
# print(response.content)

In [52]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    MessagesPlaceholder,
                                    PromptTemplate)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory,ConversationSummaryMemory

from langchain.globals import set_verbose
set_verbose(True)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

In [ ]:
# Define the system message and human message template
sys_message = SystemMessage(content='''The following is a friendly conversation between a human and an AI.  
If the AI does not know the answer to a question, it truthfully says it does not know.''')

hum_message_template = HumanMessagePromptTemplate.from_template(template='{question}')

message_history = MessagesPlaceholder(variable_name='message_log')

In [ ]:
# Create the chat prompt template
chat_template = ChatPromptTemplate.from_messages([
    sys_message,
    message_history,
    hum_message_template
])

In [ ]:
# Initialize memory (using ConversationSummaryMemory as in your original)
chat_memory = ConversationSummaryMemory(
    llm=chat,
    memory_key='message_log',
    return_messages=True  # Changed to True to work with MessagesPlaceholder
)

In [ ]:
# Create the chain
chain = LLMChain(
    llm=chat,
    prompt=chat_template,
    memory=chat_memory,
    verbose=True  # Optional for debugging
)

In [ ]:
# Example usage
response = chain.invoke({'question': "Can you give me an interesting fact I probably didn't know about?"})
print(response['text'])

In [ ]:
response = chain.invoke({'question': "Can you talk about rodri?"})
print(response['text'])

In [ ]:
response = chain.invoke({'question': "Can you talk about Kevin debroyne?"})
print(response['text'])

In [ ]:
# Check memory state
print(chat_memory.load_memory_variables({}))